In [6]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

In [7]:
load_dotenv()

True

In [8]:
model = ChatOpenAI(model="gpt-4o-mini")

In [9]:
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a helpful assistant. Answer all questions to the best of your ability."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

ai_msg = chain.invoke(
    {
        "messages": [
            HumanMessage(content="Can you explain Dynamic Programming under 50 words?")
        ],
    }
)

print(ai_msg.content)

Dynamic Programming is a method for solving complex problems by breaking them down into simpler subproblems, solving each subproblem just once, and storing their solutions. It is often used for optimization problems where overlapping subproblems and optimal substructure properties exist.


In [12]:
## Method 1: Pass Chat History to your LLM as a context.

chat_history = [
    HumanMessage(content="Can you explain Dynamic Programming under 50 words?"),
    AIMessage(content=ai_msg.content)
]

chat_history

[HumanMessage(content='Can you explain Dynamic Programming under 50 words?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Dynamic Programming is a method for solving complex problems by breaking them down into simpler subproblems, solving each subproblem just once, and storing their solutions. It is often used for optimization problems where overlapping subproblems and optimal substructure properties exist.', additional_kwargs={}, response_metadata={})]

In [19]:
def interrogate_model(chat_history: list, preserve_chat_history=True):
    current_message = [HumanMessage(content="What did I ask you earlier?")]
    chat_messages = []

    if preserve_chat_history:
        chat_messages = chat_history + current_message
    else:
        chat_messages = current_message


    ai_msg = chain.invoke(
        {
            "messages": chat_messages
        }
    )

    return ai_msg.content

In [21]:
interrogate_model(chat_history, preserve_chat_history=False)

"I'm sorry, but I don't have the ability to recall past conversations or remember any previous interactions. Each session is treated independently. How can I assist you today?"

In [ ]:
# Method 2: Trimming off the Chat History